In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torch
train_on_gpu = torch.cuda.is_available()
print(train_on_gpu)
device = torch.device("cuda:0")
device

True


device(type='cuda', index=0)

In [2]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
X_train, X_val= train_test_split(
trainset, test_size=0.2, random_state=42)

trainloader = torch.utils.data.DataLoader(X_train, batch_size=4,
                                          shuffle=True, num_workers=2)
validationloader = torch.utils.data.DataLoader(X_val, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:

model_fe = models.alexnet(pretrained=True)

# We don't calculate the gradient for the actual fully-connected layer
for param in model_fe.parameters():
    param.requires_grad = False

# We get the number of input features of the fully-connected layer
num_ftrs = model_fe.classifier[6].in_features

# Adding a new fully connected linear layer
model_fe.classifier[6] = nn.Sequential (
    nn.Linear(num_ftrs, 300),
    nn.LeakyReLU(),
    nn.Linear(300, 10),
    nn.Softmax())

In [4]:
import torch.optim as optim
model_fe = model_fe.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fe.parameters(), lr=0.001)

decayRate = 0.96
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_fe(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        writer.add_scalar('Model1 Loss/train', loss.item(), epoch*10000+i) 
        running_loss += loss.item()
        
        print('[%d, %5d] loss: %.3f %d porcent correct ' %
                  (epoch + 1, i + 1, running_loss / 2000, 0))#val_loss))
        if i % 2000 == 1999:    # print every 2000 mini-batches
            #val_loss = validate(model_fe, 1, epoch)
            print('[%d, %5d] loss: %.3f %d porcent correct ' %
                  (epoch + 1, i + 1, running_loss / 2000, 0))#val_loss))
            running_loss = 0.0
    scheduler.step()

print('Finished Training')

RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small